# 1.3 Getting ready to run code on the cluster#

### IMPORTANT: Please make sure that your are using the bash kernel to run this notebook.

Now that you can submit jobs like any self-respecting Unix ninja, you are ready to start analyzing data! Here you will learn about how to organize your research directory and setup the cluster environment to access all software you wish to use.

## Organizing your research as a pro##

This is a really nice paper with guidelines on organizing computational projects in an organized and snazzy fashion: (http://journals.plos.org/ploscompbiol/article?id=10.1371/journal.pcbi.1000424). Let's see this in action!

First, define a variable with the training camp directory

In [1]:
export TCDIR=/srv/scratch/training_camp/tc2016/$(whoami)
echo $TCDIR

/srv/scratch/training_camp/tc2016/user23


Organize your folder into subdirectories as a pro: 

In [2]:
cd ${TCDIR}
mkdir data src
ls

mkdir: cannot create directory ‘data’: File exists
mkdir: cannot create directory ‘src’: File exists
analysis  data	src  testGithub  tmp


## Preparing to run code on the cluster ## 

Our data processing will use multiple software tools. To be able to access them, we can load their paths into our session, by loading their respective modules.

To load a module, you can type
**module load [desiredModule]** - this is going to modify your path

Once a module is loaded, you can use the code associated with that module directly. For instance, let's say you want to load a module for BEDTools (a software package we will be using in this training camp). If you run:

In [3]:
source /etc/profile.d/modules.sh

In [4]:
module load bedtools/2.17.0

it loads the bedtools code, such that when you are ready to use the code, you can just directly call commands. Note that the -h or --help arguments can often be used to give help about a particular tool.


In [5]:
bedtools -h

bedtools: flexible tools for genome arithmetic and DNA sequence analysis.
usage:    bedtools <subcommand> [options]

The bedtools sub-commands include:

[ Genome arithmetic ]
    intersect     Find overlapping intervals in various ways.
    window        Find overlapping intervals within a window around an interval.
    closest       Find the closest, potentially non-overlapping interval.
    coverage      Compute the coverage over defined intervals.
    map           Apply a function to a column for each overlapping interval.
    genomecov     Compute the coverage over an entire genome.
    merge         Combine overlapping/nearby intervals into a single interval.
    cluster       Cluster (but don't merge) overlapping/nearby intervals.
    complement    Extract intervals _not_ represented by an interval file.
    subtract      Remove intervals based on overlaps b/w two files.
    slop          Adjust the size of intervals.
    flank         Create new intervals from the flanks of exi

Don't worry, you do not need to know off the top of your head the names of the modules you want. To see all software modules available on the AWS cluster, type:

In [6]:
module avail


------------------------- /usr/share/modules/versions --------------------------
3.2.10

------------------------ /usr/share/modules/modulefiles ------------------------
MACS2/2.0.9       fastqc/0.10.1     module-info       r/3.0.2
bedtools/2.17.0   homer/default     modules           samtools/0.1.19
bowtie/2.1.0      java/latest       null              ucsc_tools/2.7.2
dot               module-git        picard-tools/1.95 use.own


## The .bashrc file (=your friend) ##

In [7]:
#Where is .bashrc?
#In our home directory
cd ~
pwd
ls
#But this doesn't show bashrc...
ls -ah #this does (shows all hidden files)
#.bash_logout automatically runs things when you log out

/home/user23
training_camp
.   .bash_history  .bashrc  .config  .emacs.d  .kshenv	.login	  .python-eggs
..  .bash_logout   .cache   .cshrc   .ipython  .local	.profile  training_camp


Wouldn't it be nice to have everything ready to run when you log into the cluster?
To avoid having to run module load commands every time you log in, you can add these commands to a .bashrc file, located in your home directory. The .bashrc file contains a set of commands that get executed every time you log into the server. In this way, every time you log in, you will be all set to run all code you wish.

Note: Technically, the ~/.bashrc file is not what's executed on login; it's ~/.bash_profile, which in turn calls ~/.bashrc. If your .bash_profile does not call .bashrc, put the line source ~/.bashrc in your .bash_profile. The difference between the two files is explained here: http://www.joshstaiger.org/archives/2005/07/bash_profile_vs.html

Let's add all our desired module loading commands into a .bashrc file. 

In [8]:
bowtie_load='module load bowtie/2.1.0'
bedtools_load='module load bedtools/2.17.0'
fastqc_load='module load fastqc/0.10.1'
java_load='module load java/latest'
picard_load='module load picard-tools/1.95'
r_load='module load r/3.0.2'
samtools_load='module load samtools/0.1.19'
ucsc_load='module load ucsc_tools/2.7.2'
macs2_load='module load MACS2/2.0.9'
homer_load='module load homer/default'

#naive thing:
#NOTE: ~ is a shortcut for your home directory
#echo $bowtie_load >> ~/.bashrc #this might clutter up our bashrc if we run it a bunch of times

#only add the module load commands to the ~/.bashrc file if they don't exist in this file already 
#The || acts like an OR; it executes the command on the right if the command on the left errors out
#grep -E acts like search
#reminder: "$bowtie_load" decodes to 'module load bowtie/2.1.0'
grep -E "$bowtie_load" ~/.bashrc || echo $bowtie_load >> ~/.bashrc
grep -E "$bedtools_load" ~/.bashrc || echo $bedtools_load >> ~/.bashrc
grep -E "$fastqc_load" ~/.bashrc || echo $fastqc_load >> ~/.bashrc
grep -E "$java_load" ~/.bashrc || echo $java_load >> ~/.bashrc
grep -E "$picard_load" ~/.bashrc || echo $picard_load >> ~/.bashrc
grep -E "$r_load" ~/.bashrc || echo $r_load >> ~/.bashrc
grep -E "$samtools_load" ~/.bashrc || echo $samtools_load >> ~/.bashrc
grep -E "$ucsc_load" ~/.bashrc || echo $ucsc_load >> ~/.bashrc
grep -E "$macs2_load" ~/.bashrc || echo $macs2_load >> ~/.bashrc
grep -E "$homer_load" ~/.bashrc || echo $homer_load >> ~/.bashrc

module load bowtie/2.1.0
module load bedtools/2.17.0
module load fastqc/0.10.1
module load java/latest
module load picard-tools/1.95
module load r/3.0.2
module load samtools/0.1.19
module load ucsc_tools/2.7.2
module load MACS2/2.0.9
module load homer/default


## Defining shortcuts in the .bashrc file ##

Why stop here? You can make all your dreams come true in the .bashrc file!
For instance, you can add to the .bashrc file some shortcuts to your directories of interest, which you can then seamlessly use. Add the following to your .bashrc:

In [9]:
grep -E "shortcuts_defined" ~/.bashrc || 
(echo '#shortcuts_defined:' >> ~/.bashrc &&
echo 'export SUNETID="$(whoami)"' >> ~/.bashrc &&
echo 'export WORK_DIR="/srv/scratch/training_camp/tc2016/${SUNETID}"' >> ~/.bashrc &&
echo 'export DATA_DIR="${WORK_DIR}/data"' >> ~/.bashrc &&
echo '[[ ! -d ${DATA_DIR} ]] && mkdir "${DATA_DIR}"' >> ~/.bashrc &&
echo 'export SRC_DIR="${WORK_DIR}/src/training_camp/src"' >> ~/.bashrc &&
echo '[[ ! -d ${WORK_DIR}/src ]] && mkdir -p "${WORK_DIR}/src"' >> ~/.bashrc &&
echo 'export ANALYSIS_DIR=${WORK_DIR}/analysis' >> ~/.bashrc &&
echo '[[ ! -d ${WORK_DIR}/analysis ]] && mkdir -p "${WORK_DIR}/analysis"' >> ~/.bashrc &&
echo 'export AK_DATA_DIR="/srv/scratch/training_camp/data/fastqs"' >> ~/.bashrc &&
echo 'export AK_TOOL_DIR="/srv/scratch/training_camp/tools"' >> ~/.bashrc &&
echo 'export AK_TOOL_DIR="/srv/scratch/training_camp/metadata"' >> ~/.bashrc &&
echo 'export YEAST_DIR="/srv/scratch/training_camp/saccer3/seq"' >> ~/.bashrc &&
echo 'export YEAST_INDEX="/srv/scratch/training_camp/saccer3/bowtie2_index/saccer3"' >> ~/.bashrc &&
echo 'export YEAST_CHR="/srv/scratch/training_camp/saccer3/sacCer3.chrom.sizes"' >> ~/.bashrc &&
echo 'export TMP="${WORK_DIR}/tmp"' >> ~/.bashrc &&
echo 'export TEMP=$TMP' >> ~/.bashrc &&
echo 'export TMPDIR=$TMP' >> ~/.bashrc &&
echo 'mkdir $TMP' >> ~/.bashrc)

#shortcuts_defined:


**\$\{WORK\_DIR\}** is your main work directory

**\$\{SRC\_DIR\}** is your src/ directory

**\$\{DATA\_DIR\}** is your data/ directory

**\$\{AK\_DATA\_DIR\}** is the directory with the fastq files (the data you will use in this project)

To see your ~/.bashrc and ~/.bash_profile files in action, logout and log in again. All modules should be loaded and all shortcuts should be set!

Since logging in/out would disrupt this tutorial, we execute the commands in our ipython notebook:

In [3]:
export SUNETID="$(whoami)"
export WORK_DIR="/srv/scratch/training_camp/tc2016/${SUNETID}"
export DATA_DIR="${WORK_DIR}/data"
[[ ! -d ${DATA_DIR} ]] && mkdir "${DATA_DIR}"
export SRC_DIR="${WORK_DIR}/src/training_camp/src"
[[ ! -d ${WORK_DIR}/src ]] && mkdir -p "${WORK_DIR}/src"
export ANALYSIS_DIR=${WORK_DIR}/analysis/
[[ ! -d ${WORK_DIR}/analysis ]] && mkdir -p "${WORK_DIR}/analysis"
export AK_DATA_DIR="/srv/scratch/training_camp/data/fastqs"
export AK_METADATA_DIR="/srv/scratch/training_camp/metadata"
export AK_TOOL_DIR="/srv/scratch/training_camp/tools"
export YEAST_DIR="/srv/scratch/training_camp/saccer3/seq"
export YEAST_INDEX="/srv/scratch/training_camp/saccer3/bowtie2_index/saccer3"
export YEAST_CHR="/srv/scratch/training_camp/saccer3/sacCer3.chrom.sizes"
export TMP="${WORK_DIR}/tmp"
export TEMP=$TMP 
export TMPDIR=$TMP
mkdir $TMP

mkdir: cannot create directory ‘/srv/scratch/training_camp/tc2016/user23/tmp’: File exists


## Data and code for this project ##

Create a directory to store FASTQ files

In [5]:
mkdir -p ${DATA_DIR}/fastq #makes a directory to contain the symbolic links

In [4]:
echo ${DATA_DIR}/fastq

/srv/scratch/training_camp/tc2016/user23/data/fastq


If you wanted to analyze all the data, you would make symbolic links to the data (symbolic links are pointers to other files; they behave like copies of the original file but without actually duplicating the data):

In [29]:
#DO NOT UNCOMMENT THIS LINE!
rm ${DATA_DIR}/fastq/*
#ln -s -r ${AK_DATA_DIR}/* ${DATA_DIR}/fastq/  #creates the symbolic links (-r is needed to handle directories)

However, there is quite a bit of data, so instead, you can copy just the samples that you generated to your FASTQ directory. To do this, uncomment the line below corresponding to the samples you worked on. 

In [6]:
# cp $AK_DATA_DIR/*Ct_1_S22* $DATA_DIR/fastq
# cp $AK_DATA_DIR/*Ct_2_S23* $DATA_DIR/fastq
# cp $AK_DATA_DIR/*Ct_300_S3* $DATA_DIR/fastq
 cp $AK_DATA_DIR/*Ct_3_S24* $DATA_DIR/fastq
# cp $AK_DATA_DIR/*Ct_800_S9* $DATA_DIR/fastq


# cp $AK_DATA_DIR/*Cz_1_S16* $DATA_DIR/fastq
# cp $AK_DATA_DIR/*Cz_2_S17* $DATA_DIR/fastq
# cp $AK_DATA_DIR/*Cz_300_S1* $DATA_DIR/fastq
# cp $AK_DATA_DIR/*Cz_3_S18* $DATA_DIR/fastq
# cp $AK_DATA_DIR/*Cz_800_S7* $DATA_DIR/fastq

# cp $AK_DATA_DIR/*DMSO_1_S31* $DATA_DIR/fastq
# cp $AK_DATA_DIR/*DMSO_1_S6* $DATA_DIR/fastq
# cp $AK_DATA_DIR/*DMSO_2_S12* $DATA_DIR/fastq
# cp $AK_DATA_DIR/*DMSO_2_S32* $DATA_DIR/fastq

# cp $AK_DATA_DIR/*It_1_S25* $DATA_DIR/fastq
# cp $AK_DATA_DIR/*It_2_s26* $DATA_DIR/fastq
# cp $AK_DATA_DIR/*It_300_S5* $DATA_DIR/fastq
# cp $AK_DATA_DIR/*It_3_S27* $DATA_DIR/fastq
# cp $AK_DATA_DIR/*It_800_S11* $DATA_DIR/fastq

# cp $AK_DATA_DIR/*Kt_1_S13* $DATA_DIR/fastq
# cp $AK_DATA_DIR/*Kt_2_S14* $DATA_DIR/fastq
# cp $AK_DATA_DIR/*Kt_3_S15* $DATA_DIR/fastq
# cp $AK_DATA_DIR/*Kz_300_S4* $DATA_DIR/fastq
# cp $AK_DATA_DIR/*Kz_800_S10* $DATA_DIR/fastq

# cp $AK_DATA_DIR/*Kt_1_S13* $DATA_DIR/fastq
# cp $AK_DATA_DIR/*Kt_2_S14* $DATA_DIR/fastq
# cp $AK_DATA_DIR/*Kt_3_S15* $DATA_DIR/fastq
# cp $AK_DATA_DIR/*Kz_300_S4* $DATA_DIR/fastq
# cp $AK_DATA_DIR/*Kz_800_S10* $DATA_DIR/fastq

# cp $AK_DATA_DIR/*Mz_1_S19* $DATA_DIR/fastq
# cp $AK_DATA_DIR/*Mz_2_S20* $DATA_DIR/fastq
# cp $AK_DATA_DIR/*Mz_300_S2* $DATA_DIR/fastq
# cp $AK_DATA_DIR/*Mz_3_S21* $DATA_DIR/fastq
# cp $AK_DATA_DIR/*Mz_800_S8* $DATA_DIR/fastq

# cp $AK_DATA_DIR/*U_1_S28* $DATA_DIR/fastq
# cp $AK_DATA_DIR/*U_2_S29* $DATA_DIR/fastq
# cp $AK_DATA_DIR/*U_3_S30* $DATA_DIR/fastq

In [7]:
ls $DATA_DIR/fastq

Ct_3_S24_R1.fastq.gz  Ct_3_S24_R2.fastq.gz


In [ ]:
rm $DATA_DIR/fastq/*

Get the code we provide for this project:

In [27]:
cd ${WORK_DIR}/src
rm -r training_camp
git clone https://github.com/kundajelab/training_camp

Cloning into 'training_camp'...
remote: Counting objects: 488, done.        
remote: Compressing objects: 100% (217/217), done.        
remote: Total 488 (delta 136), reused 0 (delta 0), pack-reused 261        
Receiving objects: 100% (488/488), 4.98 MiB | 3.70 MiB/s, done.
Resolving deltas: 100% (229/229), done.
Checking connectivity... done.


## Writing code and code version control ##

Your code is precious! For this reason, you want to: 1) make sure it is backed up, and 2) keep a "diary" of all versions of your code so that you can track its progress and revert back to previous versions if need be. Both of these goals can be achieved with version control software, such as Github.

In this training camp you will get familiarized with GitHub.

First, go to Github, make an account and create a repository. Let's assume you named the repository myExampleRepo.

Then, to get used to GitHub, you can try to make a test script and add it to your repository on GitHub.

First, we will get a copy of your Github repository in your directory

In [12]:
cd ${WORK_DIR}
mkdir testGithub
cd testGithub
pwd

mkdir: cannot create directory ‘testGithub’: File exists
/srv/scratch/training_camp/tc2016/user23/testGithub


In [13]:
export githubusername="AvantiShri"
git clone https://github.com/$githubusername/myExampleRepo
ls

Cloning into 'myExampleRepo'...
Checking connectivity... done.
myExampleRepo


You should now see a directory named myExampleRepo in the current working directory. We will now make a new file and add it to the repository. We add some text to this script using the "echo" command.

In [14]:
cd myExampleRepo
touch ${SUNETID}_testscript.sh
echo "Here is some example text" >> ${SUNETID}_testscripts.sh

In [18]:
git status

On branch master

Initial commit

Untracked files:
  (use "git add <file>..." to include in what will be committed)

	user23_testscript.sh
	user23_testscripts.sh

nothing added to commit but untracked files present (use "git add" to track)


Now, you will learn how to "push" this script to Github.

First, get the most current version of the code. Since the version on your computer is already the most up to date, this wouldn't do anything. However, in general it is a smart thing to do BEFORE you make any edits to the code.

In [16]:
#git pull

fatal: Couldn't find remote ref master
Unexpected end of command stream


You will probably be told that your repository is already up-to-date. However, if you are collaborating with other people, or have modified the code from a different computer, it is very important to always do a git pull before submitting your changes.

Next, configure your github credentials. You will only need to do this once:

In [22]:
git config --global user.name "AvantiShri"
git config --global user.email "avanti@stanford.edu"

Next, add your code to the Github:

In [20]:
git add ${SUNETID}_testscript.sh

Tell Github that you are ready to load your code (so-called commit).

In [23]:
git commit -m "my first script on github"

[master (root-commit) 2bc19b7] my first script on github
 1 file changed, 0 insertions(+), 0 deletions(-)
 create mode 100644 user23_testscript.sh


Then, push your code to Github.

In [24]:
git push

Git 2.0 from 'matching' to 'simple'. To squelch this message
and maintain the current behavior after the default changes, use:

  git config --global push.default matching

To squelch this message and adopt the new behavior now, use:

  git config --global push.default simple

When push.default is set to 'matching', git will push local branches
to the remote branches that already exist with the same name.

In Git 2.0, Git will default to the more conservative 'simple'
behavior, which only pushes the current branch to the corresponding
remote branch that 'git pull' uses to update the current branch.

See 'git help config' and search for 'push.default' for further information.
(the 'simple' mode was introduced in Git 1.7.11. Use the similar mode
'current' instead of 'simple' if you sometimes use older versions of Git)

Username for 'https://github.com': 


When you visit the Github page for your code, you should be able to see your code!

These are only the most basic commands. For more on Github see https://training.github.com/kit/downloads/github-git-cheat-sheet.pdf

Let the analysis begin!